## <font color=#6B49F5> A Simple Implementation of FedAvg with PyTorch on IID Data </font> 
Please see https://towardsdatascience.com/federated-learning-a-simple-implementation-of-fedavg-federated-averaging-with-pytorch-90187c9c9577 for more details.

In [42]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import random
import math
from torch.utils.tensorboard import SummaryWriter
from matplotlib import pyplot

from pathlib import Path
import requests
import pickle
import gzip

import torch
import math
import torch.nn.functional as F
from torch import nn
from torch import optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from imblearn.over_sampling import SMOTE
import copy
from sklearn.metrics import confusion_matrix

pd.options.display.float_format = "{:,.4f}".format
sm = SMOTE(random_state=42)

In [43]:
THREAT_TYPE = 'threat_type'
THREAT_HL = 'threat_hl'

learning_rate = 0.01
numEpoch = 20
batch_size = 32
momentum = 0.9
print_amount=5
number_of_slices = 5
isSmote = False

file_name = "kdd-slice-specific-attack-cumulative-centralized.txt"
file = open(file_name, "w")

data_path = "D:\\learning\\PyTorch\\NSL_KDD-master\\"

colnames = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land',
            'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
            'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files',
            'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate',
            'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate',
            'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
            'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
            'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
            'dst_host_srv_rerror_rate', 'threat_type']

In [44]:
df_train = pd.read_csv(data_path + "KDDTrain+.csv", header = None)
df_train = df_train.iloc[:, :-1]

df_test = pd.read_csv(data_path + "KDDTest+.csv", header = None)
df_test = df_test.iloc[:, :-1]

df_train.columns = colnames
df_test.columns = colnames

df_train.loc[(df_train['threat_type'] == 'back'), 'threat_type'] = 1
df_train.loc[(df_train['threat_type'] == 'buffer_overflow'), 'threat_type'] = 2
df_train.loc[(df_train['threat_type'] == 'ftp_write'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'guess_passwd'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'imap'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'ipsweep'), 'threat_type'] = 4
df_train.loc[(df_train['threat_type'] == 'land'), 'threat_type'] = 1
df_train.loc[(df_train['threat_type'] == 'loadmodule'), 'threat_type'] = 2
df_train.loc[(df_train['threat_type'] == 'multihop'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'neptune'), 'threat_type'] = 1
df_train.loc[(df_train['threat_type'] == 'nmap'), 'threat_type'] = 4
df_train.loc[(df_train['threat_type'] == 'perl'), 'threat_type'] = 2
df_train.loc[(df_train['threat_type'] == 'phf'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'pod'), 'threat_type'] = 1
df_train.loc[(df_train['threat_type'] == 'portsweep'), 'threat_type'] = 4
df_train.loc[(df_train['threat_type'] == 'rootkit'), 'threat_type'] = 2
df_train.loc[(df_train['threat_type'] == 'satan'), 'threat_type'] = 4
df_train.loc[(df_train['threat_type'] == 'smurf'), 'threat_type'] = 1
df_train.loc[(df_train['threat_type'] == 'spy'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'teardrop'), 'threat_type'] = 1
df_train.loc[(df_train['threat_type'] == 'warezclient'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'warezmaster'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'normal'), 'threat_type'] = 0
df_train.loc[(df_train['threat_type'] == 'unknown'), 'threat_type'] = 6

df_test.loc[(df_test['threat_type'] == 'back'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'buffer_overflow'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'ftp_write'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'guess_passwd'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'imap'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'ipsweep'), 'threat_type'] = 4
df_test.loc[(df_test['threat_type'] == 'land'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'loadmodule'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'multihop'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'neptune'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'nmap'), 'threat_type'] = 4
df_test.loc[(df_test['threat_type'] == 'perl'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'phf'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'pod'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'portsweep'), 'threat_type'] = 4
df_test.loc[(df_test['threat_type'] == 'rootkit'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'satan'), 'threat_type'] = 4
df_test.loc[(df_test['threat_type'] == 'smurf'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'spy'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'teardrop'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'warezclient'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'warezmaster'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'normal'), 'threat_type'] = 0
df_test.loc[(df_test['threat_type'] == 'unknown'), 'threat_type'] = 6
df_test.loc[(df_test['threat_type'] == 'mscan'), 'threat_type'] = 4
df_test.loc[(df_test['threat_type'] == 'apache2'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'snmpgetattack'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'processtable'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'httptunnel'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'ps'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'snmpguess'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'mailbomb'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'named'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'sendmail'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'xterm'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'xlock'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'xsnoop'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'sqlattack'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'udpstorm'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'saint'), 'threat_type'] = 4
df_test.loc[(df_test['threat_type'] == 'worm'), 'threat_type'] = 1

df_full = pd.concat([df_train, df_test])

print('Attack types in full set: \n', df_full[THREAT_TYPE].value_counts())

Attack types in full set: 
 0    77053
1    53387
4    14077
3     3880
2      119
Name: threat_type, dtype: int64


In [45]:

print('Before normalization shape of data set : ', df_full.shape)
threat_type_df = df_full['threat_type'].copy()
# Considering numerical columns
# 34 numerical columns are considered for training
numerical_colmanes = ['duration', 'src_bytes', 'dst_bytes', 'wrong_fragment', 'urgent', 'hot',
                      'num_failed_logins', 'num_compromised', 'root_shell', 'su_attempted', 'num_root',
                      'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'count',
                      'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
                      'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
                      'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
                      'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
                      'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']

numerical_df_full = df_full[numerical_colmanes].copy()
print(numerical_df_full.shape)
#
# # Lets remove the numerical columns with constant value
numerical_df_full = numerical_df_full.loc[:, (numerical_df_full != numerical_df_full.iloc[0]).any()]
#
# # lets scale the values for each column from [0,1]
# # N.B. we dont have any negative values]
final_df_full = numerical_df_full / numerical_df_full.max()
print(final_df_full.shape)

df_normalized = pd.concat([final_df_full, threat_type_df], axis=1)
print('After normalization shape of data set: ', df_normalized.shape)
print(df_normalized[THREAT_TYPE].value_counts())

Before normalization shape of data set :  (148516, 42)
(148516, 34)
(148516, 33)
After normalization shape of data set:  (148516, 34)
0    77053
1    53387
4    14077
3     3880
2      119
Name: threat_type, dtype: int64


In [46]:
df_normal = df_normalized[df_normalized['threat_type'] == 0]
df_dos = df_normalized[df_normalized['threat_type'] == 1]
df_u2r = df_normalized[df_normalized['threat_type'] == 2]
df_r2u = df_normalized[df_normalized['threat_type'] == 3]
df_probe = df_normalized[df_normalized['threat_type'] == 4]

print(df_normal.shape, df_dos.shape, df_u2r.shape, df_r2u.shape, df_probe.shape)

(77053, 34) (53387, 34) (119, 34) (3880, 34) (14077, 34)


In [47]:
def divide_train_test(df, propotion=0.1):
    
    df_train = []
    df_test = []
    for key,val in df[THREAT_TYPE].value_counts().iteritems():
        df_part = df[df['threat_type'] == key]
        df_test.append(df_part[0: int(df_part.shape[0]*propotion)])
        df_train.append(df_part[int(df_part.shape[0]*propotion):df_part.shape[0]])
        
    return df_train,df_test
    

In [48]:
def get_data_for_slices(df_train, number_of_slices, isSmote=False, x_name="x_train", y_name="y_train"):
    df_normal_s1 = df_normal[int(df_normal.shape[0]*0/number_of_slices):int(df_normal.shape[0]*(1)/number_of_slices)]
    df_normal_s2 = df_normal[int(df_normal.shape[0]*1/number_of_slices):int(df_normal.shape[0]*(2)/number_of_slices)]
    df_normal_s3 = df_normal[int(df_normal.shape[0]*2/number_of_slices):int(df_normal.shape[0]*(3)/number_of_slices)]
    df_normal_s4 = df_normal[int(df_normal.shape[0]*3/number_of_slices):int(df_normal.shape[0]*(4)/number_of_slices)]
    df_normal_s5 = df_normal[int(df_normal.shape[0]*4/number_of_slices):int(df_normal.shape[0]*(5)/number_of_slices)]
    
    df_dos_s2 = df_dos[int(df_dos.shape[0]*0/4):int(df_dos.shape[0]*(1)/4)]
    df_dos_s3 = df_dos[int(df_dos.shape[0]*1/4):int(df_dos.shape[0]*(2)/4)]
    df_dos_s4 = df_dos[int(df_dos.shape[0]*2/4):int(df_dos.shape[0]*(3)/4)]
    df_dos_s5 = df_dos[int(df_dos.shape[0]*3/4):int(df_dos.shape[0]*(4)/4)]
    
    df_probe_s3 = df_probe[int(df_probe.shape[0]*0/3):int(df_probe.shape[0]*(1)/3)]
    df_probe_s4 = df_probe[int(df_probe.shape[0]*1/3):int(df_probe.shape[0]*(2)/3)]
    df_probe_s5 = df_probe[int(df_probe.shape[0]*2/3):int(df_probe.shape[0]*(3)/3)]

    df_r2u_s4 = df_r2u[int(df_r2u.shape[0]*0/2):int(df_r2u.shape[0]*(1)/2)]
    df_r2u_s5 = df_r2u[int(df_r2u.shape[0]*1/2):int(df_r2u.shape[0]*(2)/2)]
    
    df_u2r_s5 = df_u2r 
    
    df_s5 = pd.concat([df_normal_s5, df_dos_s5, df_probe_s5, df_r2u_s5, df_u2r_s5])
    df_s4 = pd.concat([df_normal_s4, df_dos_s4, df_probe_s4, df_r2u_s4])
    df_s3 = pd.concat([df_normal_s3, df_dos_s3, df_probe_s3])
    df_s2 = pd.concat([df_normal_s2,df_dos_s2])
    df_s1 = df_normal_s1

    x_data_dict= dict()
    y_data_dict= dict()

    y_s1 = (df_s1.pop('threat_type').values).astype('int')
    unique, counts = np.unique(y_s1, return_counts=True)
    print('slice 1: ')
    print(dict(zip(unique, counts)))
    y_info1 = torch.tensor(y_s1).type(torch.LongTensor)
    x_info1 = torch.tensor(df_s1.values).float()

    y_s2 = (df_s2.pop('threat_type').values).astype('int')
    unique, counts = np.unique(y_s2, return_counts=True)
    print('slice 2: ')
    print(dict(zip(unique, counts)))
    y_info2 = torch.tensor(y_s2).type(torch.LongTensor)
    x_info2 = torch.tensor(df_s2.values).float()

    y_s3 = (df_s3.pop('threat_type').values).astype('int')
    unique, counts = np.unique(y_s3, return_counts=True)
    print('slice 3: ')
    print( dict(zip(unique, counts)))
    y_info3 = torch.tensor(y_s3).type(torch.LongTensor)
    x_info3 = torch.tensor(df_s3.values).float()

    y_s4 = (df_s4.pop('threat_type').values).astype('int')
    unique, counts = np.unique(y_s4, return_counts=True)
    print('slice 4: ')
    print(dict(zip(unique, counts)))
    y_info4 = torch.tensor(y_s4).type(torch.LongTensor)
    x_info4 = torch.tensor(df_s4.values).float()

    y_s5 = (df_s5.pop('threat_type').values).astype('int')
    unique, counts = np.unique(y_s5, return_counts=True)
    print('slice 5: ')
    print(dict(zip(unique, counts)))
    y_info5 = torch.tensor(y_s5).type(torch.LongTensor)
    x_info5 = torch.tensor(df_s5.values).float()


    x_data_dict.update({"x_train0" : x_info1})
    y_data_dict.update({"y_train0" : y_info1})

    x_data_dict.update({"x_train1" : x_info2})
    y_data_dict.update({"y_train1" : y_info2})

    x_data_dict.update({"x_train2" : x_info3})
    y_data_dict.update({"y_train2" : y_info3})

    x_data_dict.update({"x_train3" : x_info4})
    y_data_dict.update({"y_train3" : y_info4})

    x_data_dict.update({"x_train4" : x_info5})
    y_data_dict.update({"y_train4" : y_info5})
    
    return x_data_dict, y_data_dict 
        
        

In [49]:
df_train, df_test = divide_train_test(df_normalized,propotion=0.1)
# print('Value counts in train set : ')
# df_train[THREAT_TYPE].value_counts()
# print('Value counts in test set : ')
# print(df_test[THREAT_TYPE].value_counts())

x_train_dict, y_train_dict = get_data_for_slices(df_train, number_of_slices, isSmote)

for y_train_s in y_train_dict:
    print(y_train_s)
df_test = pd.concat(df_test)
y_test = df_test.pop(THREAT_TYPE).values
x_test = df_test.values

print('Test set size is : x => ' + str(x_test.shape) + ' y => ' + str(y_test.shape))
x_test = torch.tensor(x_test).float()
y_test = torch.tensor(y_test.astype('int')).type(torch.LongTensor)

inputs = x_test.shape[1]
outputs = 5

slice 1: 
{0: 15410}
slice 2: 
{0: 15411, 1: 13346}
slice 3: 
{0: 15410, 1: 13347, 4: 4692}
slice 4: 
{0: 15411, 1: 13347, 3: 1940, 4: 4692}
slice 5: 
{0: 15411, 1: 13347, 2: 119, 3: 1940, 4: 4693}
y_train0
y_train1
y_train2
y_train3
y_train4
Test set size is : x => (14849, 33) y => (14849,)


--------------------------
### <span style="background-color:#F087F9"> Classification Model </span> 

In [50]:
class Net2nn(nn.Module):
    def __init__(self, inputs, outputs):
        super(Net2nn, self).__init__()
        self.fc1=nn.Linear(inputs,200)
        self.fc2=nn.Linear(200,200)
        self.fc3=nn.Linear(200,outputs)
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x

In [51]:
class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

In [52]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    correct = 0

    for data, target in train_loader:
        output = model(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        prediction = output.argmax(dim=1, keepdim=True)
        correct += prediction.eq(target.view_as(prediction)).sum().item()
        

    return train_loss / len(train_loader), correct/len(train_loader.dataset)

In [53]:
def validation(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            
            test_loss += criterion(output, target).item()
            prediction = output.argmax(dim=1, keepdim=True)
            correct += prediction.eq(target.view_as(prediction)).sum().item()

    test_loss /= len(test_loader)
    correct /= len(test_loader.dataset)

    return (test_loss, correct)

In [54]:
def confusion_mat(model, test_loader, desc = 'none'):
    
    print(desc)
    y_pred = []
    y_true = []

    # iterate over test data
    for inputs, labels in test_loader:
        output = model(inputs)  # Feed Network

        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output)  # Save Prediction

        labels = labels.data.cpu().numpy()
        y_true.extend(labels)  # Save Truth

    cf_matrix = confusion_matrix(y_true, y_pred)
#     df_cm = pd.DataFrame(cf_matrix, index=[i for i in Counter(y_test)],
#                          columns=[i for i in Counter(y_test)])
#     plt.figure(1)
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')
#     plt.figure(figsize=(12, 7))

#     sn.heatmap(df_cm, annot=True).set(xlabel='Predicted label', ylabel='True label')
#     plt.savefig('D:\\learning\\PyTorch\\experiment\\cf\\cf_fl_'+str(self.number_of_slices)+'.png')
    print('confusion matrix for normal scenario for slices : ' + str(number_of_slices))
    print(cf_matrix)
    file.write('\ncf matrix for slice :' + str(number_of_slices))
    file.write('\n'+str(cf_matrix))

---------------------------------
### <span style="background-color:#F087F9"> Functions for Federated Averaging </span> 

In [55]:
def create_model_optimizer_criterion_dict(number_of_slices):
    model_dict = dict()
    optimizer_dict= dict()
    criterion_dict = dict()
    
    for i in range(number_of_slices):
        model_name="model"+str(i)
        model_info=Net2nn(inputs, outputs)
        model_dict.update({model_name : model_info })
        
        optimizer_name="optimizer"+str(i)
        optimizer_info = torch.optim.SGD(model_info.parameters(), lr=learning_rate, momentum=momentum)
        optimizer_dict.update({optimizer_name : optimizer_info })
        
        criterion_name = "criterion"+str(i)
        criterion_info = nn.CrossEntropyLoss()
        criterion_dict.update({criterion_name : criterion_info})
        
    return model_dict, optimizer_dict, criterion_dict 

In [56]:
def get_averaged_weights(model_dict, number_of_slices):
   
    fc1_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc1.weight.shape)
    fc1_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc1.bias.shape)
    
    fc2_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc2.weight.shape)
    fc2_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc2.bias.shape)
    
    fc3_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc3.weight.shape)
    fc3_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc3.bias.shape)
    
    with torch.no_grad():
    
    
        for i in range(number_of_slices):
            fc1_mean_weight += model_dict[name_of_models[i]].fc1.weight.data.clone()
            fc1_mean_bias += model_dict[name_of_models[i]].fc1.bias.data.clone()
        
            fc2_mean_weight += model_dict[name_of_models[i]].fc2.weight.data.clone()
            fc2_mean_bias += model_dict[name_of_models[i]].fc2.bias.data.clone()
        
            fc3_mean_weight += model_dict[name_of_models[i]].fc3.weight.data.clone()
            fc3_mean_bias += model_dict[name_of_models[i]].fc3.bias.data.clone()

        
        fc1_mean_weight =fc1_mean_weight/number_of_slices
        fc1_mean_bias = fc1_mean_bias/ number_of_slices
    
        fc2_mean_weight =fc2_mean_weight/number_of_slices
        fc2_mean_bias = fc2_mean_bias/ number_of_slices
    
        fc3_mean_weight =fc3_mean_weight/number_of_slices
        fc3_mean_bias = fc3_mean_bias/ number_of_slices
    
    return fc1_mean_weight, fc1_mean_bias, fc2_mean_weight, fc2_mean_bias, fc3_mean_weight, fc3_mean_bias

In [57]:
def set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_slices):
    fc1_mean_weight, fc1_mean_bias, fc2_mean_weight, fc2_mean_bias, fc3_mean_weight, fc3_mean_bias = get_averaged_weights(model_dict, number_of_slices=number_of_slices)
    with torch.no_grad():
        main_model.fc1.weight.data = fc1_mean_weight.data.clone()
        main_model.fc2.weight.data = fc2_mean_weight.data.clone()
        main_model.fc3.weight.data = fc3_mean_weight.data.clone()

        main_model.fc1.bias.data = fc1_mean_bias.data.clone()
        main_model.fc2.bias.data = fc2_mean_bias.data.clone()
        main_model.fc3.bias.data = fc3_mean_bias.data.clone() 
    return main_model

In [58]:
def compare_local_and_merged_model_performance(number_of_slices):
    accuracy_table=pd.DataFrame(data=np.zeros((number_of_slices,3)), columns=["sample", "local_ind_model", "merged_main_model"])
    for i in range (number_of_slices):
    
        test_ds = TensorDataset(x_test, y_test)
        test_dl = DataLoader(test_ds, batch_size=batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        individual_loss, individual_accuracy = validation(model, test_dl, criterion)
        main_loss, main_accuracy =validation(main_model, test_dl, main_criterion )
    
        accuracy_table.loc[i, "sample"]="sample "+str(i)
        accuracy_table.loc[i, "local_ind_model"] = individual_accuracy
        accuracy_table.loc[i, "merged_main_model"] = main_accuracy

    return accuracy_table

In [59]:
def send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_slices):
    with torch.no_grad():
        for i in range(number_of_slices):
            print('Updating model :' + name_of_models[i] )
            model_dict[name_of_models[i]].fc1.weight.data =main_model.fc1.weight.data.clone()
            model_dict[name_of_models[i]].fc2.weight.data =main_model.fc2.weight.data.clone()
            model_dict[name_of_models[i]].fc3.weight.data =main_model.fc3.weight.data.clone() 
            
            model_dict[name_of_models[i]].fc1.bias.data =main_model.fc1.bias.data.clone()
            model_dict[name_of_models[i]].fc2.bias.data =main_model.fc2.bias.data.clone()
            model_dict[name_of_models[i]].fc3.bias.data =main_model.fc3.bias.data.clone() 
    
    return model_dict

In [60]:
def start_train_end_node_process(number_of_slices):
    for i in range (number_of_slices): 

        print('Federated learning for slice '+ str(i+1))
        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

#         valid_ds = TensorDataset(x_valid_dict[name_of_x_valid_sets[i]], y_valid_dict[name_of_y_valid_sets[i]])
#         valid_dl = DataLoader(valid_ds, batch_size=batch_size * 2)
        
        test_ds = TensorDataset(x_test_dict[name_of_x_test_sets[i]], y_test_dict[name_of_y_test_sets[i]])
        test_dl = DataLoader(test_ds, batch_size= batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        print("Subset" ,i)
        for epoch in range(numEpoch):        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
#             valid_loss, valid_accuracy = validation(model, valid_dl, criterion)
            test_loss, test_accuracy = validation(model, test_dl, criterion)
    
            print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.5f}".format(train_accuracy) + " | test accuracy: {:7.5f}".format(test_accuracy))

In [61]:

def start_train_end_node_process_without_print(number_of_slices):
    for i in range (number_of_slices): 

        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

        test_ds = TensorDataset(x_test, y_test)
        test_dl = DataLoader(test_ds, batch_size= batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        for epoch in range(numEpoch):        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
            test_loss, test_accuracy = validation(model, test_dl, criterion)
            
        confusion_mat(model, test_dl, desc = 'Confusion matrix for ' + name_of_models[i])

In [62]:
def start_train_end_node_process_print_some(number_of_slices, print_amount):
    for i in range (number_of_slices): 
        
        print('Federated learning for slice '+ str(i+1))
        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], 
                                 y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

        test_ds = TensorDataset(x_test, y_test)
        test_dl = DataLoader(test_ds, batch_size= batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        if i<print_amount:
            print("Subset" ,i)
            
        for epoch in range(numEpoch):
        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
            test_loss, test_accuracy = validation(model, test_dl, criterion)
            
            if i<print_amount:        
                print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.5f}".format(train_accuracy) + " | test accuracy: {:7.5f}".format(test_accuracy)) 
                
        confusion_mat(model, test_dl, desc = 'Confusion matrix for ' + name_of_models[i])

In [63]:
# x_train, y_train, x_valid, y_valid,x_test, y_test = map(torch.tensor, (x_train, y_train, x_valid, y_valid, x_test, y_test))


----------------

### <span style="background-color:#F087F9"> Let's examine what would the performance of the centralized model be if the data were not distributed to nodes at all? </span>   

The model used in this example is very simple, different things can be done to improve model performance, such as using more complex models, increasing epoch or hyperparameter tuning. However, the purpose here is to compare the performance of the main model that is formed by combining the parameters of the local models trained on their own data with a centralized model that trained on all training data. In this way, we can gain insight into the capacity of federated learning.


In [64]:
# initial_model = Net2nn()
# initial_optimizer = torch.optim.SGD(initial_model.parameters(), lr=0.01, momentum=0.9)
# initial_criterion = nn.CrossEntropyLoss()

centralized_model = Net2nn(inputs, outputs)
centralized_optimizer = torch.optim.SGD(centralized_model.parameters(), lr=0.01, momentum=0.9)
centralized_criterion = nn.CrossEntropyLoss()

In [65]:
print("------ Centralized Model - 4 ------")
file.write('Centralized model 4')

train_acc4 = []
test_acc4 = []
train_loss4 = []
test_loss4 = []

test_ds = TensorDataset(x_test, y_test)
test_dl = DataLoader(test_ds, batch_size=batch_size * 2)

centralized_model4 = Net2nn(inputs,outputs)
centralized_optimizer4 = torch.optim.SGD(centralized_model4.parameters(), lr=0.01, momentum=0.9)
centralized_criterion4 = nn.CrossEntropyLoss()
#     centralized_model = copy.deepcopy(initial_model)
#     centralized_optimizer = copy.deepcopy(initial_optimizer)
#     centralized_criterion = copy.deepcopy(initial_criterion)
x_name = 'x_train' + str(4)
y_name = 'y_train' + str(4)
train_ds = TensorDataset(x_train_dict[x_name], y_train_dict[y_name])
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

for epoch in range(numEpoch):
    central_train_loss, central_train_accuracy = train(centralized_model4, train_dl, centralized_criterion4, centralized_optimizer4)
    central_test_loss, central_test_accuracy = validation(centralized_model4, test_dl, centralized_criterion4)

    train_acc4.append(central_train_accuracy)
    train_loss4.append(central_train_loss)
    test_acc4.append(central_test_accuracy)
    test_loss4.append(central_test_loss)

#         print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
print(" | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
confusion_mat(centralized_model4, test_dl)

print(centralized_model4.fc2.weight[0:1,0:5])  
print("------ Training finished ------")
print('Mean train accuracy: ' + str(sum(train_acc4)/len(train_acc4)))
print('Mean test accuracy: ' + str(sum(test_acc4)/len(test_acc4)))



------ Centralized Model - 4 ------
 | train accuracy:  0.9393 | test accuracy:  0.9560
none
confusion matrix for normal scenario for slices : 5
[[7371  128    5  179   22]
 [  43 5288    0    3    4]
 [   2    0    6    3    0]
 [ 133   11    1  241    2]
 [  66    9    0   43 1289]]
tensor([[ 0.1070,  0.0409,  0.0785,  0.0295, -0.0291]],
       grad_fn=<SliceBackward>)
------ Training finished ------
Mean train accuracy: 0.922823148408899
Mean test accuracy: 0.9489729948144656


In [66]:
file.write('\nCentralized model 4 Mean train accuracy: ' + str(sum(train_acc4)/len(train_acc4)))
file.write('\nCentralized model 4 Mean test accuracy: ' + str(sum(test_acc4)/len(test_acc4)))

59

In [67]:
print("------ Centralized Model - 0 ------")
file.write('Centralized model 0')


train_acc0 = []
test_acc0 = []
train_loss0 = []
test_loss0 = []

test_ds = TensorDataset(x_test, y_test)
test_dl = DataLoader(test_ds, batch_size=batch_size * 2)

centralized_model0 = Net2nn(inputs, outputs)
centralized_optimizer0 = torch.optim.SGD(centralized_model0.parameters(), lr=0.01, momentum=0.9)
centralized_criterion0 = nn.CrossEntropyLoss()

#     centralized_model = copy.deepcopy(initial_model)
#     centralized_optimizer = copy.deepcopy(initial_optimizer)
#     centralized_criterion = copy.deepcopy(initial_criterion)
x_name = 'x_train' + str(0)
y_name = 'y_train' + str(0)
train_ds = TensorDataset(x_train_dict[x_name], y_train_dict[y_name])
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

for epoch in range(numEpoch):
    central_train_loss, central_train_accuracy = train(centralized_model0, train_dl, centralized_criterion0, centralized_optimizer0)
    central_test_loss, central_test_accuracy = validation(centralized_model0, test_dl, centralized_criterion0)

    train_acc0.append(central_train_accuracy)
    train_loss0.append(central_train_loss)
    test_acc0.append(central_test_accuracy)
    test_loss0.append(central_test_loss)

#         print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
print(" | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
confusion_mat(centralized_model0, test_dl)

print(centralized_model0.fc2.weight[0:1,0:5])  
print("------ Training finished ------")
print('Mean train accuracy: ' + str(sum(train_acc0)/len(train_acc0)))
print('Mean test accuracy: ' + str(sum(test_acc0)/len(test_acc0)))


------ Centralized Model - 0 ------
 | train accuracy:  1.0000 | test accuracy:  0.5189
none
confusion matrix for normal scenario for slices : 5
[[7705    0    0    0    0]
 [5338    0    0    0    0]
 [  11    0    0    0    0]
 [ 388    0    0    0    0]
 [1407    0    0    0    0]]
tensor([[-0.0120, -0.0431, -0.0116, -0.0685, -0.0401]],
       grad_fn=<SliceBackward>)
------ Training finished ------
Mean train accuracy: 0.9996885139519792
Mean test accuracy: 0.5188901609535995


In [68]:
file.write('\nCentralized model 0 Mean train accuracy: ' + str(sum(train_acc0)/len(train_acc0)))
file.write('\nCentralized model 0 Mean test accuracy: ' + str(sum(test_acc0)/len(test_acc0)))

59

In [69]:
print("------ Centralized Model - 1 ------")
file.write('Centralized model 1')


train_acc1 = []
test_acc1 = []
train_loss1 = []
test_loss1 = []

test_ds = TensorDataset(x_test, y_test)
test_dl = DataLoader(test_ds, batch_size=batch_size * 2)

centralized_model1 = Net2nn(inputs, outputs)
centralized_optimizer1 = torch.optim.SGD(centralized_model1.parameters(), lr=0.01, momentum=0.9)
centralized_criterion1 = nn.CrossEntropyLoss()

#     centralized_model = copy.deepcopy(initial_model)
#     centralized_optimizer = copy.deepcopy(initial_optimizer)
#     centralized_criterion = copy.deepcopy(initial_criterion)
x_name = 'x_train' + str(1)
y_name = 'y_train' + str(1)
train_ds = TensorDataset(x_train_dict[x_name], y_train_dict[y_name])
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

for epoch in range(numEpoch):
    central_train_loss, central_train_accuracy = train(centralized_model1, train_dl, centralized_criterion1, centralized_optimizer1)
    central_test_loss, central_test_accuracy = validation(centralized_model1, test_dl, centralized_criterion1)

    train_acc1.append(central_train_accuracy)
    train_loss1.append(central_train_loss)
    test_acc1.append(central_test_accuracy)
    test_loss1.append(central_test_loss)

#         print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
print(" | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
confusion_mat(centralized_model1, test_dl)

print(centralized_model1.fc2.weight[0:1,0:5])  
print("------ Training finished ------")
print('Mean train accuracy: ' + str(sum(train_acc1)/len(train_acc1)))
print('Mean test accuracy: ' + str(sum(test_acc1)/len(test_acc1)))


------ Centralized Model - 1 ------
 | train accuracy:  0.9908 | test accuracy:  0.8710
none
confusion matrix for normal scenario for slices : 5
[[7662   43    0    0    0]
 [  66 5272    0    0    0]
 [  11    0    0    0    0]
 [ 386    2    0    0    0]
 [1018  389    0    0    0]]
tensor([[0.0015, 0.0656, 0.0372, 0.0246, 0.0230]], grad_fn=<SliceBackward>)
------ Training finished ------
Mean train accuracy: 0.9861859721111381
Mean test accuracy: 0.8674355175432688


In [70]:
file.write('\nCentralized model 1 Mean train accuracy: ' + str(sum(train_acc1)/len(train_acc1)))
file.write('\nCentralized model 1 Mean test accuracy: ' + str(sum(test_acc1)/len(test_acc1)))

59

In [71]:
print("------ Centralized Model - 2 ------")
file.write('Centralized model 2')

train_acc2 = []
test_acc2 = []
train_loss2 = []
test_loss2 = []

test_ds = TensorDataset(x_test, y_test)
test_dl = DataLoader(test_ds, batch_size=batch_size * 2)

centralized_model2 = Net2nn(inputs, outputs)
centralized_optimizer2 = torch.optim.SGD(centralized_model2.parameters(), lr=0.01, momentum=0.9)
centralized_criterion2 = nn.CrossEntropyLoss()

#     centralized_model = copy.deepcopy(initial_model)
#     centralized_optimizer = copy.deepcopy(initial_optimizer)
#     centralized_criterion = copy.deepcopy(initial_criterion)
x_name = 'x_train' + str(2)
y_name = 'y_train' + str(2)
train_ds = TensorDataset(x_train_dict[x_name], y_train_dict[y_name])
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

for epoch in range(numEpoch):
    central_train_loss, central_train_accuracy = train(centralized_model2, train_dl, centralized_criterion2, centralized_optimizer2)
    central_test_loss, central_test_accuracy = validation(centralized_model2, test_dl, centralized_criterion2)

    train_acc2.append(central_train_accuracy)
    train_loss2.append(central_train_loss)
    test_acc2.append(central_test_accuracy)
    test_loss2.append(central_test_loss)

#         print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
print(" | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
confusion_mat(centralized_model2, test_dl)

print(centralized_model2.fc2.weight[0:1,0:5])  
print("------ Training finished ------")
print('Mean train accuracy: ' + str(sum(train_acc2)/len(train_acc2)))
print('Mean test accuracy: ' + str(sum(test_acc2)/len(test_acc2)))


------ Centralized Model - 2 ------
 | train accuracy:  0.9841 | test accuracy:  0.9595
none
confusion matrix for normal scenario for slices : 5
[[7644   39    0    0   22]
 [  69 5269    0    0    0]
 [  11    0    0    0    0]
 [ 382    4    0    0    2]
 [  71    1    0    0 1335]]
tensor([[ 0.0141,  0.0130, -0.0097,  0.0012,  0.0330]],
       grad_fn=<SliceBackward>)
------ Training finished ------
Mean train accuracy: 0.9773416245627671
Mean test accuracy: 0.953518755471749


In [72]:
file.write('\nCentralized model 2 Mean train accuracy: ' + str(sum(train_acc2)/len(train_acc2)))
file.write('\nCentralized model 2 Mean test accuracy: ' + str(sum(test_acc2)/len(test_acc2)))

58

In [73]:
print("------ Centralized Model - 3 ------")
file.write('Centralized model 3')

train_acc3 = []
test_acc3 = []
train_loss3 = []
test_loss3 = []

test_ds = TensorDataset(x_test, y_test)
test_dl = DataLoader(test_ds, batch_size=batch_size * 2)

centralized_model3 = Net2nn(inputs, outputs)
centralized_optimizer3 = torch.optim.SGD(centralized_model3.parameters(), lr=0.01, momentum=0.9)
centralized_criterion3 = nn.CrossEntropyLoss()

#     centralized_model = copy.deepcopy(initial_model)
#     centralized_optimizer = copy.deepcopy(initial_optimizer)
#     centralized_criterion = copy.deepcopy(initial_criterion)
x_name = 'x_train' + str(3)
y_name = 'y_train' + str(3)
train_ds = TensorDataset(x_train_dict[x_name], y_train_dict[y_name])
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

for epoch in range(numEpoch):
    central_train_loss, central_train_accuracy = train(centralized_model3, train_dl, centralized_criterion3, centralized_optimizer3)
    central_test_loss, central_test_accuracy = validation(centralized_model3, test_dl, centralized_criterion3)

    train_acc3.append(central_train_accuracy)
    train_loss3.append(central_train_loss)
    test_acc3.append(central_test_accuracy)
    test_loss3.append(central_test_loss)

#         print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
print(" | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
confusion_mat(centralized_model3, test_dl)

print(centralized_model3.fc2.weight[0:1,0:5])  
print("------ Training finished ------")
print('Mean train accuracy: ' + str(sum(train_acc3)/len(train_acc3)))
print('Mean test accuracy: ' + str(sum(test_acc3)/len(test_acc3)))


------ Centralized Model - 3 ------
 | train accuracy:  0.9647 | test accuracy:  0.9747
none
confusion matrix for normal scenario for slices : 5
[[7593   29    0   63   20]
 [ 101 5231    0    2    4]
 [   9    0    0    2    0]
 [  53    1    0  333    1]
 [  78    1    0   11 1317]]
tensor([[-0.0251,  0.0431, -0.0508, -0.0543,  0.0314]],
       grad_fn=<SliceBackward>)
------ Training finished ------
Mean train accuracy: 0.9510921164170669
Mean test accuracy: 0.9681763081689004


In [74]:
file.write('\nCentralized model 3 Mean train accuracy: ' + str(sum(train_acc3)/len(train_acc3)))
file.write('\nCentralized model 3 Mean test accuracy: ' + str(sum(test_acc3)/len(test_acc3)))

59


----------------
-----------------
**Data is distributed to nodes**

<!-- ### <span style="background-color:#F087F9"> Datanın nodelara dağıtılması </span>    -->

In [26]:
print(x_train_dict["x_train1"].shape, y_train_dict["y_train1"].shape)
print(x_test.shape, y_test.shape)

torch.Size([28757, 33]) torch.Size([28757])
torch.Size([14849, 33]) torch.Size([14849])


**Main model is created**

In [27]:
main_model = Net2nn(inputs,outputs)
main_optimizer = torch.optim.SGD(main_model.parameters(), lr=learning_rate, momentum=0.9)
main_criterion = nn.CrossEntropyLoss()

**Models,optimizers and loss functions in nodes are defined**

In [28]:
model_dict, optimizer_dict, criterion_dict = create_model_optimizer_criterion_dict(number_of_slices)

**Keys of dicts are being made iterable**

In [29]:
name_of_x_train_sets=list(x_train_dict.keys())
name_of_y_train_sets=list(y_train_dict.keys())

name_of_models=list(model_dict.keys())
name_of_optimizers=list(optimizer_dict.keys())
name_of_criterions=list(criterion_dict.keys())

print(name_of_x_train_sets)
print(name_of_y_train_sets)
print("\n ------------")
print(name_of_models)
print(name_of_optimizers)
print(name_of_criterions)

['x_train0', 'x_train1', 'x_train2', 'x_train3', 'x_train4']
['y_train0', 'y_train1', 'y_train2', 'y_train3', 'y_train4']

 ------------
['model0', 'model1', 'model2', 'model3', 'model4']
['optimizer0', 'optimizer1', 'optimizer2', 'optimizer3', 'optimizer4']
['criterion0', 'criterion1', 'criterion2', 'criterion3', 'criterion4']


In [30]:
print(main_model.fc2.weight[0:1,0:5])
print(model_dict["model1"].fc2.weight[0:1,0:5])

tensor([[ 5.8793e-05,  6.4562e-02,  1.4268e-02,  4.8174e-02, -7.9750e-03]],
       grad_fn=<SliceBackward>)
tensor([[ 0.0160, -0.0007,  0.0480,  0.0423,  0.0411]],
       grad_fn=<SliceBackward>)


**Parameters of main model are sent to nodes**  
Since the parameters of the main model and parameters of all local models in the nodes are randomly initialized, all these parameters will be different from each other. For this reason, the main model sends its parameters to the nodes before the training of local models in the nodes begins. You can check the weights below.

In [31]:
model_dict=send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_slices)

Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4


In [32]:
print(main_model.fc2.weight[0:1,0:5])
print(model_dict["model1"].fc2.weight[0:1,0:5])
print(model_dict["model0"].fc2.weight[0,0:5])
print(model_dict["model1"].fc2.weight[0,0:5])
print(model_dict["model2"].fc2.weight[0,0:5])
print(model_dict["model3"].fc2.weight[0,0:5])
print(model_dict["model4"].fc2.weight[0,0:5])

tensor([[ 5.8793e-05,  6.4562e-02,  1.4268e-02,  4.8174e-02, -7.9750e-03]],
       grad_fn=<SliceBackward>)
tensor([[ 5.8793e-05,  6.4562e-02,  1.4268e-02,  4.8174e-02, -7.9750e-03]],
       grad_fn=<SliceBackward>)
tensor([ 5.8793e-05,  6.4562e-02,  1.4268e-02,  4.8174e-02, -7.9750e-03],
       grad_fn=<SliceBackward>)
tensor([ 5.8793e-05,  6.4562e-02,  1.4268e-02,  4.8174e-02, -7.9750e-03],
       grad_fn=<SliceBackward>)
tensor([ 5.8793e-05,  6.4562e-02,  1.4268e-02,  4.8174e-02, -7.9750e-03],
       grad_fn=<SliceBackward>)
tensor([ 5.8793e-05,  6.4562e-02,  1.4268e-02,  4.8174e-02, -7.9750e-03],
       grad_fn=<SliceBackward>)
tensor([ 5.8793e-05,  6.4562e-02,  1.4268e-02,  4.8174e-02, -7.9750e-03],
       grad_fn=<SliceBackward>)


**Models in the nodes are trained**

In [33]:
# start_train_end_node_process()
start_train_end_node_process_print_some(number_of_slices, print_amount)

Federated learning for slice 1
Subset 0
epoch:   1 | train accuracy: 0.99377 | test accuracy: 0.51889
epoch:   2 | train accuracy: 1.00000 | test accuracy: 0.51889
epoch:   3 | train accuracy: 1.00000 | test accuracy: 0.51889
epoch:   4 | train accuracy: 1.00000 | test accuracy: 0.51889
epoch:   5 | train accuracy: 1.00000 | test accuracy: 0.51889
epoch:   6 | train accuracy: 1.00000 | test accuracy: 0.51889
epoch:   7 | train accuracy: 1.00000 | test accuracy: 0.51889
epoch:   8 | train accuracy: 1.00000 | test accuracy: 0.51889
epoch:   9 | train accuracy: 1.00000 | test accuracy: 0.51889
epoch:  10 | train accuracy: 1.00000 | test accuracy: 0.51889
epoch:  11 | train accuracy: 1.00000 | test accuracy: 0.51889
epoch:  12 | train accuracy: 1.00000 | test accuracy: 0.51889
epoch:  13 | train accuracy: 1.00000 | test accuracy: 0.51889
epoch:  14 | train accuracy: 1.00000 | test accuracy: 0.51889
epoch:  15 | train accuracy: 1.00000 | test accuracy: 0.51889
epoch:  16 | train accuracy: 1

In [34]:
## As you can see, wieghts of local models are updated after training process
print(main_model.fc2.weight[0,0:5])
print(model_dict["model0"].fc2.weight[0,0:5])
print(model_dict["model1"].fc2.weight[0,0:5])
print(model_dict["model2"].fc2.weight[0,0:5])
print(model_dict["model3"].fc2.weight[0,0:5])
print(model_dict["model4"].fc2.weight[0,0:5])

tensor([ 5.8793e-05,  6.4562e-02,  1.4268e-02,  4.8174e-02, -7.9750e-03],
       grad_fn=<SliceBackward>)
tensor([ 4.0280e-05,  5.9772e-02,  1.2785e-02,  4.8027e-02, -8.4644e-03],
       grad_fn=<SliceBackward>)
tensor([-0.0201,  0.0795, -0.0594,  0.0590, -0.0076], grad_fn=<SliceBackward>)
tensor([-0.0333,  0.0723, -0.0424,  0.0709,  0.0131], grad_fn=<SliceBackward>)
tensor([-0.0932,  0.0275, -0.1628,  0.0558, -0.0642], grad_fn=<SliceBackward>)
tensor([-0.0026,  0.1306,  0.0116,  0.0001,  0.0475], grad_fn=<SliceBackward>)


### Let's compare the performance of federated main model, individual local models and centralized model  

**Federated main model vs individual local models before 1st iteration (on distributed test set)**  
Since main model is randomly initialized and no action taken on it yet, its performance is very poor. Please before_acc_table

In [35]:
before_acc_table=compare_local_and_merged_model_performance(number_of_slices=number_of_slices)
before_test_loss, before_test_accuracy = validation(main_model, test_dl, main_criterion)
file.write('\nbefore training main model')
confusion_mat(main_model, test_dl)

main_model= set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_slices) 
print(main_model.fc2.weight[0,0:5])
after_acc_table=compare_local_and_merged_model_performance(number_of_slices=number_of_slices)
after_test_loss, after_test_accuracy = validation(main_model, test_dl, main_criterion)
file.write('\nafter training main model')
confusion_mat(main_model, test_dl)

none
confusion matrix for normal scenario for slices : 5
[[   0 7705    0    0    0]
 [   0 5338    0    0    0]
 [   0   11    0    0    0]
 [   0  388    0    0    0]
 [   0 1387    0    0   20]]
tensor([-0.0298,  0.0739, -0.0480,  0.0468, -0.0039], grad_fn=<SliceBackward>)
none
confusion matrix for normal scenario for slices : 5
[[7705    0    0    0    0]
 [ 319 5019    0    0    0]
 [  11    0    0    0    0]
 [ 388    0    0    0    0]
 [ 861  164    0    0  382]]


In [36]:
print("Federated main model vs individual local models before FedAvg first iteration")
file.write('\nBefore training federated')
file.write('\n'+str(before_acc_table))
before_acc_table

Federated main model vs individual local models before FedAvg first iteration


,sample,local_ind_model,merged_main_model
0,sample 0,0.5189,0.3608
1,sample 1,0.8708,0.3608
2,sample 2,0.9578,0.3608
3,sample 3,0.9667,0.3608
4,sample 4,0.9599,0.3608


In [37]:
print("Federated main model vs individual local models after FedAvg first iteration")
file.write('\nAfter training federated')
file.write('\n'+str(after_acc_table))
after_acc_table

Federated main model vs individual local models after FedAvg first iteration


,sample,local_ind_model,merged_main_model
0,sample 0,0.5189,0.8826
1,sample 1,0.8708,0.8826
2,sample 2,0.9578,0.8826
3,sample 3,0.9667,0.8826
4,sample 4,0.9599,0.8826


**Federated main model vs centralized model before 1st iteration (on all test data)**  
Please be aware that the centralized model gets approximately %98 accuracy on all test data.

In [38]:
print("Before 1st iteration main model accuracy on all test data: {:7.4f}".format(before_test_accuracy))
print("After 1st iteration main model accuracy on all test data: {:7.4f}".format(after_test_accuracy))
print("Centralized model accuracy on all test data: {:7.4f}".format(central_test_accuracy))

Before 1st iteration main model accuracy on all test data:  0.3608
After 1st iteration main model accuracy on all test data:  0.8826
Centralized model accuracy on all test data:  0.9560


This is a single iteration, we can send the weights of the main model back to the nodes and repeat the above steps.
Now let's check how the performance of the main model improves when we repeat the iteration 10 more times.

In [39]:
for i in range(100):
    model_dict=send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_slices)
    start_train_end_node_process_print_some(number_of_slices, print_amount)
    main_model= set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_slices) 
    test_loss, test_accuracy = validation(main_model, test_dl, main_criterion)
    print("Iteration", str(i+2), ": main_model accuracy on all test data: {:7.4f}".format(test_accuracy))   

Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4
Federated learning for slice 1
Subset 0
epoch:   1 | train accuracy: 1.00000 | test accuracy: 0.86430
epoch:   2 | train accuracy: 1.00000 | test accuracy: 0.86006
epoch:   3 | train accuracy: 1.00000 | test accuracy: 0.85669
epoch:   4 | train accuracy: 1.00000 | test accuracy: 0.85305
epoch:   5 | train accuracy: 1.00000 | test accuracy: 0.84881
epoch:   6 | train accuracy: 1.00000 | test accuracy: 0.84450
epoch:   7 | train accuracy: 1.00000 | test accuracy: 0.83743
epoch:   8 | train accuracy: 1.00000 | test accuracy: 0.83447
epoch:   9 | train accuracy: 1.00000 | test accuracy: 0.83332
epoch:  10 | train accuracy: 1.00000 | test accuracy: 0.83265
epoch:  11 | train accuracy: 1.00000 | test accuracy: 0.83117
epoch:  12 | train accuracy: 1.00000 | test accuracy: 0.82955
epoch:  13 | train accuracy: 1.00000 | test accuracy: 0.82814
epoch:  14 | train accuracy: 1.00000 | 

epoch:   7 | train accuracy: 1.00000 | test accuracy: 0.88289
epoch:   8 | train accuracy: 1.00000 | test accuracy: 0.88181
epoch:   9 | train accuracy: 1.00000 | test accuracy: 0.88134
epoch:  10 | train accuracy: 1.00000 | test accuracy: 0.88087
epoch:  11 | train accuracy: 1.00000 | test accuracy: 0.88053
epoch:  12 | train accuracy: 1.00000 | test accuracy: 0.88019
epoch:  13 | train accuracy: 1.00000 | test accuracy: 0.87986
epoch:  14 | train accuracy: 1.00000 | test accuracy: 0.87966
epoch:  15 | train accuracy: 1.00000 | test accuracy: 0.87945
epoch:  16 | train accuracy: 1.00000 | test accuracy: 0.87918
epoch:  17 | train accuracy: 1.00000 | test accuracy: 0.87891
epoch:  18 | train accuracy: 1.00000 | test accuracy: 0.87858
epoch:  19 | train accuracy: 1.00000 | test accuracy: 0.87831
epoch:  20 | train accuracy: 1.00000 | test accuracy: 0.87811
Confusion matrix for model0
confusion matrix for normal scenario for slices : 5
[[7705    0    0    0    0]
 [ 468 4870    0    0   

epoch:  16 | train accuracy: 1.00000 | test accuracy: 0.88309
epoch:  17 | train accuracy: 1.00000 | test accuracy: 0.88282
epoch:  18 | train accuracy: 1.00000 | test accuracy: 0.88221
epoch:  19 | train accuracy: 1.00000 | test accuracy: 0.88181
epoch:  20 | train accuracy: 1.00000 | test accuracy: 0.88154
Confusion matrix for model0
confusion matrix for normal scenario for slices : 5
[[7705    0    0    0    0]
 [ 481 4857    0    0    0]
 [  11    0    0    0    0]
 [ 388    0    0    0    0]
 [ 879    0    0    0  528]]
Federated learning for slice 2
Subset 1
epoch:   1 | train accuracy: 0.99332 | test accuracy: 0.93367
epoch:   2 | train accuracy: 0.99360 | test accuracy: 0.92565
epoch:   3 | train accuracy: 0.99284 | test accuracy: 0.92444
epoch:   4 | train accuracy: 0.99384 | test accuracy: 0.92437
epoch:   5 | train accuracy: 0.99357 | test accuracy: 0.92437
epoch:   6 | train accuracy: 0.99378 | test accuracy: 0.92161
epoch:   7 | train accuracy: 0.99388 | test accuracy: 0.9

epoch:   1 | train accuracy: 0.99343 | test accuracy: 0.93804
epoch:   2 | train accuracy: 0.99426 | test accuracy: 0.93380
epoch:   3 | train accuracy: 0.99433 | test accuracy: 0.93804
epoch:   4 | train accuracy: 0.99391 | test accuracy: 0.93387
epoch:   5 | train accuracy: 0.99458 | test accuracy: 0.92835
epoch:   6 | train accuracy: 0.99468 | test accuracy: 0.93003
epoch:   7 | train accuracy: 0.99510 | test accuracy: 0.92255
epoch:   8 | train accuracy: 0.99461 | test accuracy: 0.92222
epoch:   9 | train accuracy: 0.99485 | test accuracy: 0.92774
epoch:  10 | train accuracy: 0.99454 | test accuracy: 0.92700
epoch:  11 | train accuracy: 0.99447 | test accuracy: 0.92323
epoch:  12 | train accuracy: 0.99464 | test accuracy: 0.91966
epoch:  13 | train accuracy: 0.99499 | test accuracy: 0.92262
epoch:  14 | train accuracy: 0.99513 | test accuracy: 0.92323
epoch:  15 | train accuracy: 0.99416 | test accuracy: 0.92141
epoch:  16 | train accuracy: 0.99475 | test accuracy: 0.92626
epoch:  

epoch:  10 | train accuracy: 0.99496 | test accuracy: 0.93104
epoch:  11 | train accuracy: 0.99503 | test accuracy: 0.93434
epoch:  12 | train accuracy: 0.99513 | test accuracy: 0.93266
epoch:  13 | train accuracy: 0.99496 | test accuracy: 0.93340
epoch:  14 | train accuracy: 0.99538 | test accuracy: 0.92855
epoch:  15 | train accuracy: 0.99503 | test accuracy: 0.92814
epoch:  16 | train accuracy: 0.99510 | test accuracy: 0.92902
epoch:  17 | train accuracy: 0.99513 | test accuracy: 0.93124
epoch:  18 | train accuracy: 0.99503 | test accuracy: 0.92707
epoch:  19 | train accuracy: 0.99548 | test accuracy: 0.92545
epoch:  20 | train accuracy: 0.99524 | test accuracy: 0.92457
Confusion matrix for model1
confusion matrix for normal scenario for slices : 5
[[7671   34    0    0    0]
 [  25 5313    0    0    0]
 [  11    0    0    0    0]
 [ 387    1    0    0    0]
 [ 649   13    0    0  745]]
Federated learning for slice 3
Subset 2
epoch:   1 | train accuracy: 0.99010 | test accuracy: 0.9

...

epoch:  13 | train accuracy: 0.99650 | test accuracy: 0.96882
epoch:  14 | train accuracy: 0.99620 | test accuracy: 0.96936
epoch:  15 | train accuracy: 0.99629 | test accuracy: 0.96882
epoch:  16 | train accuracy: 0.99626 | test accuracy: 0.96909
epoch:  17 | train accuracy: 0.99644 | test accuracy: 0.96895
epoch:  18 | train accuracy: 0.99653 | test accuracy: 0.96882
epoch:  19 | train accuracy: 0.99608 | test accuracy: 0.96929
epoch:  20 | train accuracy: 0.99611 | test accuracy: 0.96895
Confusion matrix for model2
confusion matrix for normal scenario for slices : 5
[[7649   30    0    0   26]
 [   5 5331    0    0    2]
 [  10    0    0    0    1]
 [ 349    6    0    1   32]
 [   0    0    0    0 1407]]
Federated learning for slice 4
Subset 3
epoch:   1 | train accuracy: 0.98539 | test accuracy: 0.98963
epoch:   2 | train accuracy: 0.98570 | test accuracy: 0.98835
epoch:   3 | train accuracy: 0.98613 | test accuracy: 0.98808
epoch:   4 | train accuracy: 0.98663 | test accuracy: 0.9

confusion matrix for normal scenario for slices : 5
[[7661   28    0    0   16]
 [   5 5333    0    0    0]
 [  11    0    0    0    0]
 [ 359    5    0    1   23]
 [   4    0    0    0 1403]]
Federated learning for slice 4
Subset 3
epoch:   1 | train accuracy: 0.98522 | test accuracy: 0.98808
epoch:   2 | train accuracy: 0.98624 | test accuracy: 0.98835
epoch:   3 | train accuracy: 0.98680 | test accuracy: 0.98902
epoch:   4 | train accuracy: 0.98638 | test accuracy: 0.98943
epoch:   5 | train accuracy: 0.98689 | test accuracy: 0.98835
epoch:   6 | train accuracy: 0.98689 | test accuracy: 0.99044
epoch:   7 | train accuracy: 0.98613 | test accuracy: 0.98828
epoch:   8 | train accuracy: 0.98647 | test accuracy: 0.98983
epoch:   9 | train accuracy: 0.98672 | test accuracy: 0.99050
epoch:  10 | train accuracy: 0.98703 | test accuracy: 0.99098
epoch:  11 | train accuracy: 0.98661 | test accuracy: 0.99172
epoch:  12 | train accuracy: 0.98672 | test accuracy: 0.98997
epoch:  13 | train accu

epoch:   6 | train accuracy: 0.98678 | test accuracy: 0.98990
epoch:   7 | train accuracy: 0.98666 | test accuracy: 0.98821
epoch:   8 | train accuracy: 0.98638 | test accuracy: 0.98882
epoch:   9 | train accuracy: 0.98683 | test accuracy: 0.98976
epoch:  10 | train accuracy: 0.98689 | test accuracy: 0.98862
epoch:  11 | train accuracy: 0.98734 | test accuracy: 0.98997
epoch:  12 | train accuracy: 0.98680 | test accuracy: 0.98781
epoch:  13 | train accuracy: 0.98672 | test accuracy: 0.99024
epoch:  14 | train accuracy: 0.98630 | test accuracy: 0.99003
epoch:  15 | train accuracy: 0.98672 | test accuracy: 0.99118
epoch:  16 | train accuracy: 0.98748 | test accuracy: 0.99118
epoch:  17 | train accuracy: 0.98686 | test accuracy: 0.99064
epoch:  18 | train accuracy: 0.98686 | test accuracy: 0.98821
epoch:  19 | train accuracy: 0.98720 | test accuracy: 0.99064
epoch:  20 | train accuracy: 0.98655 | test accuracy: 0.98949
Confusion matrix for model3
confusion matrix for normal scenario for s

epoch:  15 | train accuracy: 0.98680 | test accuracy: 0.98889
epoch:  16 | train accuracy: 0.98712 | test accuracy: 0.99071
epoch:  17 | train accuracy: 0.98720 | test accuracy: 0.98902
epoch:  18 | train accuracy: 0.98697 | test accuracy: 0.99125
epoch:  19 | train accuracy: 0.98737 | test accuracy: 0.98613
epoch:  20 | train accuracy: 0.98692 | test accuracy: 0.99017
Confusion matrix for model3
confusion matrix for normal scenario for slices : 5
[[7619   25    0   47   14]
 [   8 5327    0    3    0]
 [   5    0    2    4    0]
 [  15    0    0  371    2]
 [   9    0    0   14 1384]]
Federated learning for slice 5
Subset 4
epoch:   1 | train accuracy: 0.97401 | test accuracy: 0.98155
epoch:   2 | train accuracy: 0.97640 | test accuracy: 0.97919
epoch:   3 | train accuracy: 0.97558 | test accuracy: 0.97946
epoch:   4 | train accuracy: 0.97677 | test accuracy: 0.97246
epoch:   5 | train accuracy: 0.97677 | test accuracy: 0.98047
epoch:   6 | train accuracy: 0.97649 | test accuracy: 0.9

epoch:   1 | train accuracy: 0.97480 | test accuracy: 0.98135
epoch:   2 | train accuracy: 0.97747 | test accuracy: 0.97643
epoch:   3 | train accuracy: 0.97694 | test accuracy: 0.97993
epoch:   4 | train accuracy: 0.97654 | test accuracy: 0.97899
epoch:   5 | train accuracy: 0.97651 | test accuracy: 0.97959
epoch:   6 | train accuracy: 0.97649 | test accuracy: 0.97811
epoch:   7 | train accuracy: 0.97756 | test accuracy: 0.97677
epoch:   8 | train accuracy: 0.97736 | test accuracy: 0.97333
epoch:   9 | train accuracy: 0.97685 | test accuracy: 0.97576
epoch:  10 | train accuracy: 0.97643 | test accuracy: 0.97751
epoch:  11 | train accuracy: 0.97708 | test accuracy: 0.97751
epoch:  12 | train accuracy: 0.97657 | test accuracy: 0.97832
epoch:  13 | train accuracy: 0.97663 | test accuracy: 0.97508
epoch:  14 | train accuracy: 0.97674 | test accuracy: 0.97225
epoch:  15 | train accuracy: 0.97685 | test accuracy: 0.97205
epoch:  16 | train accuracy: 0.97750 | test accuracy: 0.97629
epoch:  

In [40]:
file = open(file_name, "a")
confusion_mat(main_model, test_dl, desc = 'Confusion matrix for after 10 rounds of the dataset')

Confusion matrix for after 10 rounds of the dataset
confusion matrix for normal scenario for slices : 5
[[7677   26    0    1    1]
 [   6 5332    0    0    0]
 [  11    0    0    0    0]
 [ 244    1    0  139    4]
 [  37    0    0    0 1370]]


The accuracy of the centralized model was calculated as approximately 98%. The accuracy of the main model obtained by FedAvg method started from 85% and improved to 94%. In this case, we can say that although the main model obtained by FedAvg method was trained without seeing the data, its performance cannot be underestimated.

In [41]:
file.close()